In [37]:
import pandas as pd

In [38]:
df_fdc = pd.read_excel('./data/raw/Sensor_Data_DSS_반출.xlsx')

동일값으로만 이루어진 열 확인

In [39]:
# 모든 값이 동일한 열과 그 값을 확인하는 함수
def find_constant_columns(df):
    constant_columns = {}
    for col in df.columns:
        if df[col].nunique() == 1:  # 고유한 값이 1개일 때, 즉 동일한 값으로만 이루어졌을 때
            constant_columns[col] = df[col].iloc[0]  # 그 열과 값을 저장
    return constant_columns

# 같은 값으로만 이루어진 열과 값을 출력
constant_columns = find_constant_columns(df_fdc)
print("같은 값으로 이루어진 열과 값:", constant_columns)

# 확인 후 원한다면 다음 코드를 실행해 제거
# cleaned_data = small_data.drop(columns=constant_columns.keys())

같은 값으로 이루어진 열과 값: {}


결측치 확인

In [40]:
# 모든 값이 NaN인 행의 개수를 확인
nan_rows_count = df_fdc.iloc[:,5:].isna().all(axis=1).sum()

print(f"모든 값이 NaN인 행의 개수: {nan_rows_count}")


모든 값이 NaN인 행의 개수: 243


In [41]:
nan_rows=df_fdc.iloc[:,5:].isna().all(axis=1)
df_fdc_remove_missing = df_fdc[~nan_rows]
df_fdc_remove_missing

,ID,EQUIP_ID,DATE_TIME,YVALUE,PASS(1)/FAIL(0),X38_05,X26_09,X30_10,X34_02,X53_02,...,X41_13,X50_13,X53_13,X57_13,X58_13.1,X36_13,X45_13,X38_13,X51_13,X34_13
0,ID_1,A,2024-01-01 22:37:51,22,1,0.121411,NaN,409.088888,8.047076,299.983923,...,101.877777,6.926000,299.625444,-346.516666,2.343333,-52640.0,0.208187,11.002221,34.194675,8.101333
1,ID_2,A,2024-01-01 22:42:09,17,1,0.061222,473598.210309,405.624999,7.883037,299.832481,...,101.662500,7.137705,299.731000,-345.600000,2.050588,-52530.0,1.682933,NaN,34.120142,7.915882
2,ID_3,A,2024-01-04 14:56:58,16,1,0.176816,485638.515282,404.300000,7.817357,299.687928,...,101.488888,7.119611,299.145111,-345.600000,2.213333,-52530.0,1.293624,NaN,33.942874,7.911555
3,ID_4,A,2024-01-04 15:05:18,11,1,0.083624,294250.798184,411.833333,7.992666,299.689037,...,101.600000,6.926000,299.145111,345.600000,2.253333,-52640.0,1.878500,10.848367,33.977119,8.106444
4,ID_5,B,2024-01-09 01:53:50,49,0,0.101859,448471.535723,403.200000,7.950222,300.177518,...,103.222222,7.038000,299.256000,344.400000,3.503333,-52800.0,2.747937,10.838689,33.745461,8.043777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,ID_1303,A,2024-07-20 12:31:35,39,1,0.071052,NaN,414.009999,8.883185,299.919074,...,100.600000,7.019333,299.256000,345.600000,2.340000,-52640.0,0.523473,11.075014,33.838124,9.031333
1303,ID_1304,B,2024-06-18 10:16:45,3,1,0.097651,NaN,407.566666,7.685615,300.068230,...,101.322222,7.038000,299.387944,343.566666,2.913333,-53405.0,2.864315,NaN,32.204434,7.819666
1306,ID_1307,B,2024-06-18 10:30:34,7,1,0.114646,NaN,407.970000,7.767259,300.136925,...,101.266666,7.038000,299.256000,-345.100000,1.723333,-53405.0,0.406210,NaN,32.192348,7.799555
1307,ID_1308,A,2024-06-11 10:24:59,11,1,0.091802,NaN,413.188888,7.984000,299.555074,...,101.100000,6.938444,299.256000,345.600000,2.140000,-52640.0,0.536421,11.223006,33.880427,8.010555


In [42]:
nan_rows_count = df_fdc_remove_missing.iloc[:,5:].isna().all(axis=1).sum()

print(f"모든 값이 NaN인 행의 개수: {nan_rows_count}")

모든 값이 NaN인 행의 개수: 0


In [43]:
# 각 열에서 결측치 개수를 확인
missing_values = df_fdc_remove_missing.isnull().sum()

# 결측치가 있는 열만 필터링
missing_values = missing_values[missing_values > 0]
print(missing_values)

X26_09      469
X51_09      469
X30_03      469
X44_06      469
X27_06      469
X59_05      469
X26_04      469
X53_04      469
X59_02      469
X55_04      851
X59_10      469
X25_03      474
X45_09      469
X44_09.1    469
X28_09      469
X27_09      469
X36_09      469
X50_09      469
X45_06      133
X45_05      139
X25_09      469
X34_09      469
X53_09      469
X57_09      469
X29_09      469
X49_04      253
X47_05      233
X35_06      239
X49_10      253
X31_05      249
X31_01      249
X35_05      239
X49_01      253
X19_10      253
X35_02      239
X8_10       239
X35_10      239
X35_03      239
X31_03      249
X49_03      253
X33_02      249
X41_05      233
X35_01      239
X31_09      500
X49_05      253
X49_06      253
X31_04      249
X31_06      249
X35_09      491
X31_10      249
X33_03      249
X49_09      502
X35_04      239
X49_02      253
X38_13      490
dtype: int64


In [44]:
# 결측치가 있는 열들을 제거
df_fdc_remove_missing = df_fdc_remove_missing.drop(columns=missing_values.index)

# 제거 후 남은 열 확인
print("제거 후 남은 열:", df_fdc_remove_missing.columns)


제거 후 남은 열: Index(['ID', 'EQUIP_ID', 'DATE_TIME', 'YVALUE', 'PASS(1)/FAIL(0)', 'X38_05',
       'X30_10', 'X34_02', 'X53_02', 'X52_10',
       ...
       'X39_13', 'X41_13', 'X50_13', 'X53_13', 'X57_13', 'X58_13.1', 'X36_13',
       'X45_13', 'X51_13', 'X34_13'],
      dtype='object', length=221)


In [45]:
# 각 열에서 결측치 개수를 확인
missing_values = df_fdc_remove_missing.isnull().sum()

# 결측치가 있는 열만 필터링
missing_values = missing_values[missing_values > 0]
print(missing_values)

Series([], dtype: int64)


In [46]:
# from sklearn.ensemble import IsolationForest

# # 5번째 열부터 시작하는 수치형 데이터만 선택 (이전과 동일)
# df_numeric = df_fdc_remove_missing.iloc[:, 5:]

# # Isolation Forest 모델 생성
# iso_forest = IsolationForest(contamination=0.05, random_state=42)  # contamination은 이상치 비율을 지정, 여기선 5%로 설정
# outlier_predictions = iso_forest.fit_predict(df_numeric.select_dtypes(include=['float64', 'int64']))

# # 이상치 탐지 결과: -1은 이상치, 1은 정상 데이터
# df_numeric['outlier'] = outlier_predictions

# # 이상치 확인 (outlier == -1인 데이터)
# outliers = df_numeric[df_numeric['outlier'] == -1]
# print(f"탐지된 이상치:\n{outliers}")

# # 이상치가 아닌 데이터만 남김 (outlier == 1인 데이터)
# df_fdc_remove_outlier_numeric = df_numeric[df_numeric['outlier'] == 1].drop(columns=['outlier'])

# # 결과 확인
# print(f"이상치 제거 후 남은 데이터:\n{df_fdc_remove_outlier_numeric}")


In [47]:
# df_final = pd.concat([df_fdc_remove_missing.iloc[:,:5], df_fdc_remove_outlier_numeric], axis=1)

# df_final = df_final[~df_final['ID'].isin(['ID_1231', 'ID_1233', 'ID_1239', 'ID_1241', 'ID_1244', 'ID_1245'])]

In [48]:
df_final = df_fdc_remove_missing
df_final = df_final[~df_final['ID'].isin(['ID_1231', 'ID_1233', 'ID_1239', 'ID_1241', 'ID_1244', 'ID_1245','ID_1309'])]

In [49]:
import os

# 이미지 폴더 경로와 DataFrame에서 파일명을 가져오는 부분
image_folder = './data/image'
image_ids = df_final['ID']  # DataFrame의 'id' 컬럼에 있는 파일명 리스트

# 이미지 파일명 리스트 생성 함수
def get_image_filenames(image_folder, image_ids):
    image_filenames = []
    for image_id in image_ids:
        image_path = os.path.join(image_folder, f"{image_id}.png")  # 확장자는 파일에 맞게 수정
        if os.path.exists(image_path):  # 파일이 존재하는지 확인
            image_filenames.append(image_path)  # 이미지 경로를 리스트에 추가
        else:
            print(f"파일을 찾을 수 없습니다: {image_id}")
    return image_filenames

# 파일명 리스트 불러오기
image_filenames = get_image_filenames(image_folder, image_ids)

# 결과 확인: 불러온 파일명 리스트 출력
print(len(image_filenames))


1059


In [50]:
import cv2
import numpy as np

def load_images(image_folder):
    images = []
    for img_path in image_filenames:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 흑백 이미지 로드
        img = cv2.resize(img, (128, 128))  # 이미지 크기 조정
        images.append(img)
    return np.array(images)

image_folder = './data/image'
wafer_images = load_images(image_folder)

In [55]:
sensor_data = df_final.iloc[:,5:]
sensor_data

,X38_05,X30_10,X34_02,X53_02,X52_10,X26_01,X36_01,X39_01,X51_10,X55_10,...,X39_13,X41_13,X50_13,X53_13,X57_13,X58_13.1,X36_13,X45_13,X51_13,X34_13
0,0.121411,409.088888,8.047076,299.983923,97730.555555,282038.263905,-22976.933333,34.196690,1.569555,179.013043,...,2.553611,101.877777,6.926000,299.625444,-346.516666,2.343333,-52640.0,0.208187,34.194675,8.101333
1,0.061222,405.624999,7.883037,299.832481,98018.750000,473591.279871,-23178.066666,34.138272,1.619000,207.542020,...,2.485882,101.662500,7.137705,299.731000,-345.600000,2.050588,-52530.0,1.682933,34.120142,7.915882
2,0.176816,404.300000,7.817357,299.687928,98218.750000,485632.131896,-22453.000000,33.956974,1.803124,209.699462,...,2.477777,101.488888,7.119611,299.145111,-345.600000,2.213333,-52530.0,1.293624,33.942874,7.911555
3,0.083624,411.833333,7.992666,299.689037,97527.777777,294244.397053,-24883.214285,33.950931,1.506333,181.173476,...,2.544166,101.600000,6.926000,299.145111,345.600000,2.253333,-52640.0,1.878500,33.977119,8.106444
4,0.101859,403.200000,7.950222,300.177518,97947.222222,448464.988731,-26053.357142,33.741432,1.498666,208.639360,...,2.519166,103.222222,7.038000,299.256000,344.400000,3.503333,-52800.0,2.747937,33.745461,8.043777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,0.061466,413.977777,8.714148,299.924481,97772.222222,582612.664849,-23841.142857,33.854239,1.575444,229.003050,...,2.536944,100.711111,7.019333,299.256000,-346.766666,2.083333,-52640.0,0.557315,33.850210,9.016111
1302,0.071052,414.009999,8.883185,299.919074,97762.500000,582619.090631,-24767.857142,33.844570,1.521000,229.004068,...,2.544166,100.600000,7.019333,299.256000,345.600000,2.340000,-52640.0,0.523473,33.838124,9.031333
1303,0.097651,407.566666,7.685615,300.068230,97961.111111,140729.416027,-22954.933333,32.198391,1.509444,119.074881,...,2.480555,101.322222,7.038000,299.387944,343.566666,2.913333,-53405.0,2.864315,32.204434,7.819666
1306,0.114646,407.970000,7.767259,300.136925,98032.500000,140736.017254,-23744.214285,32.154880,1.519000,119.076070,...,2.490000,101.266666,7.038000,299.256000,-345.100000,1.723333,-53405.0,0.406210,32.192348,7.799555


In [52]:
# 장비 센서 데이터 로드
from sklearn.preprocessing import StandardScaler

scaler =StandardScaler()
sensor_data_scaled = scaler.fit_transform(sensor_data)
df_sensor_scaled = pd.DataFrame(sensor_data_scaled, columns=sensor_data.columns)


In [56]:
df_sensor_scaled

,X38_05,X30_10,X34_02,X53_02,X52_10,X26_01,X36_01,X39_01,X51_10,X55_10,...,X39_13,X41_13,X50_13,X53_13,X57_13,X58_13.1,X36_13,X45_13,X51_13,X34_13
0,0.431704,0.505880,-0.005844,0.094083,-1.056823,-0.537424,0.178158,0.728032,0.273120,-0.131338,...,1.148349,0.606750,-1.411871,0.599236,-1.025163,0.624457,0.069690,-1.245993,0.613556,0.198906
1,-0.663939,-0.365120,-0.765624,-0.715168,0.441586,0.429478,0.004635,0.622256,0.808083,0.376130,...,-0.544824,0.398250,0.547024,0.985864,-1.022508,0.255430,0.413880,-0.010365,0.476391,-0.631428
2,1.440262,-0.698292,-1.069834,-1.487607,1.481446,0.490256,0.630168,0.293984,2.800185,0.414506,...,-0.747442,0.230103,0.379601,-1.160116,-1.022508,0.460582,0.413880,-0.336551,0.150161,-0.650801
3,-0.256147,1.195975,-0.257855,-1.481681,-2.111125,-0.475811,-1.466436,0.283042,-0.410901,-0.092908,...,0.912231,0.337717,-1.411871,-1.160116,0.979444,0.511005,0.069690,0.153492,0.213183,0.221789
4,0.075792,-0.974889,-0.454443,1.128584,0.069692,0.302648,-2.475947,-0.096293,-0.493853,0.395649,...,0.287251,1.908874,-0.375541,-0.753955,0.975969,2.086725,-0.430950,0.881957,-0.213144,-0.058794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,-0.659497,1.735198,3.083836,-0.223554,-0.840186,0.979785,-0.567416,0.107964,0.336835,0.757874,...,0.731687,-0.523191,-0.548266,-0.753955,-1.025887,0.296707,0.069690,-0.953473,-0.020371,4.294707
1055,-0.485000,1.743300,3.866765,-0.252447,-0.890734,0.979817,-1.366915,0.090457,-0.252214,0.757892,...,0.912231,-0.630804,-0.548266,-0.753955,0.979444,0.620255,0.069690,-0.981828,-0.042614,4.362862
1056,-0.000808,0.123115,-1.680024,0.544589,0.141904,-1.250708,0.197138,-2.890242,-0.377242,-1.197505,...,-0.677995,0.068683,-0.375541,-0.270673,0.973555,1.342985,-2.323995,0.979465,-3.049135,-1.062223
1057,0.308558,0.224534,-1.301874,0.911670,0.513077,-1.250675,-0.483794,-2.969027,-0.273852,-1.197484,...,-0.441877,0.014876,-0.375541,-0.753955,-1.021060,-0.157100,-2.323995,-1.080078,-3.071378,-1.152267
